In [22]:
from dotenv import load_dotenv
import os
import requests
load_dotenv('../.env')

True

In [23]:
BEARER_TOKEN = os.getenv('GITHUB_TOKEN')
if BEARER_TOKEN :
    print('GITHUB TOKEN found in envirionment')
else:
   print('GITHUB TOKEN not found')

GITHUB TOKEN found in envirionment


In [ ]:
USER = 'Kaggle' # will take input from the user instead hardcoded
BASE_URL = 'https://api.github.com/'


response = requests.get(
    f'{BASE_URL}/users/{USER}/repos',
    headers={
        'Authorization': f'Bearer {BEARER_TOKEN}'
    }
)
repositories = response.json()
repositories

In [25]:
class Repository:
    name = ''
    stargazers_count = 0
    forks_count = 0

    # further api metrics

    commits_count = 0
    contributors_count = 0
    branches_count = 0
    tags_count = 0
    issues_count = 0
    releases_count = 0
    closed_issues_count = 0
    enviroments_count = 0

    def __init__(self, name, stargazers_count, forks_count):
        self.name = name
        self.stargazers_count = stargazers_count
        self.forks_count = forks_count

        self.commits_count = self.__get_metrics_count('commits')
        self.contributors_count = self.__get_metrics_count('contributors')
        self.branches_count = self.__get_metrics_count('branches')
        self.tags_count = self.__get_metrics_count('tags')
        self.issues_count = self.__get_metrics_count('issues')
        self.releases_count = self.__get_metrics_count('releases')        
  
    def __get_metrics_count(self, url_extension):
        url = f'{BASE_URL}/repos/{USER}/{self.name}/{url_extension}'
        response = requests.get(url, headers={
            'Authorization': f'Bearer {BEARER_TOKEN}'
        })
        return len(response.json())

In [30]:
print(repositories[0]['name'])

.allstar


In [34]:
repo = Repository(repositories[1]['name'], repositories[1]['stargazers_count'], repositories[1]['forks_count'])
    
print(repo.commits_count)
print(repo.contributors_count)
print(repo.branches_count)
print(repo.tags_count)
print(repo.issues_count)
print(repo.releases_count)
print(repo.closed_issues_count)


30
6
1
0
1
0
0
